## one image predict

In [ ]:
import cv2
import numpy as np
net = cv2.dnn.readNetFromDarknet("/content/cfg_mask/yolov4-custom.cfg","/content/yolov4-custom_8000.weights")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
classes = [line.strip() for line in open("/content/cfg_mask/obj.names")]
colors = [(0,0,255),(255,0,0),(0,255,0), (255, 255, 0)] # 設定不同class 的顏色

# 回傳預測後的圖片
def yolo_detect(frame):
    # forward propogation
    # img = cv2.resize(frame, None, fx=0.4, fy=0.4)
    img = frame
    # print(image.shape)
    height, width, channels = img.shape 
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # get detection boxes
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            # print(detection)
            tx, ty, tw, th, confidence = detection[0:5]
            scores = detection[5:]
            # print(detection[5:])
            class_id = np.argmax(scores)
            # print(class_id)
            if confidence > 0.3:   
                center_x = int(tx * width)
                center_y = int(ty * height)
                w = int(tw * width)
                h = int(th * height)

                # 取得箱子方框座標
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # if x < 0:
                #     x = 0
                # if y < 0:
                #     y = 0                
                boxes.append([x, y, w, h])
                print(x, y, w, h)
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    # for i in range(len(boxes)):
    #     print(class_ids[i], boxes, sep=' ')
    # print(len(class_ids))
    # print(len(boxes))
    # draw boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            # print(i)
            x, y, w, h = boxes[i]
            # print(class_ids[i])
            label = str(classes[class_ids[i]])
            # print(label)
            # print(class_ids[i])
            # print("=================")
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y -5), font, 3, color, 3)
    return img

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt

img = cv2.imread("/content/yolo/0001.png")
# print(img.shape)
Image.open("/content/yolo/0001.png")
im = yolo_detect(img)
img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)